___
# Análise Exploratória de Dados: Pandas e Junção de conjuntos de dados
___

## Aula 02


**Links Úteis:**
1. https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
2. https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html?highlight=join#pandas.DataFrame.join

**Hoje:**
1.	Trabalhar com Pandas 

**Próxima aula:**
1.	Leitura prévia necessária: Magalhães e Lima (7ª. Edição): pág. 9 a 16 – destacando para variáveis qualitativas.


___

##  Renda per capita, Emissão de CO2 per capita e países sem litoral

Foram coletados dois conjuntos de dados do site https://www.gapminder.org/:
1. Emissão de CO2 per capita
1. Renda per capita (sendo usado PBI como uma `proxy` de renda)

Nesses *dataframes*, as linhas representam os países e as colunas representam o ano.

No *dataframe* *co2* criado a seguir, o conteúdo de cada célula é a medida de CO2 de um determinado ano (coluna) para determinado país (linha). 

No *dataframe* *income* criado a seguir, o conteúdo de cada célula é a medida de PIB per capita de um determinado ano (coluna) para um determinado país (linha). 

Também usamos as colunas *landlocked* e *region* do primeiro encontro. Neste caso não há uma coluna ano a ano porque este atributo muda só quando mudam as fronteiras de um país.


Como os arquivos estão separados, será preciso juntá-los cruzando o país.

Vamos nos restringir a analisar o ano de **2010**.

___

## Leitura dos arquivos e mudanças de índices

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import math as math
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
#Leitura dos arquivos em Excel
co2 = pd.read_excel('indicator CDIAC carbon_dioxide_emissions_per_capita.xlsx')
income = pd.read_excel('indicator gapminder gdp_per_capita_ppp.xlsx')
landlocked = pd.read_excel('landlocked_region.xlsx')

### CO2

É possível verificar que cada linha representa um país e as colunas representam o ano.


In [ ]:
co2.head()

Vamos manter apenas o ano de 2010:

In [ ]:
co2_2010 = co2.loc[:, ["CO2 per capita", 2010]]
co2_2010.head()

### Renda

O mesmo se aplica a renda.

Lembrando que GDP significa *gross domestic product*, ou *produto interno bruto* 

In [ ]:
income_2010 = income.loc[:, ["GDP per capita", 2010]]
income_2010.head()

### Landlocked e Region

Lembrando das colunas *landlocked*, que indica se um país tem litoral ou não, e *region*, que  indica a que região pertence

In [ ]:
landlocked.head()

___
# Inner Join

Vamos agora juntar as três tabelas, selecionar apenas o ano de 2010 e remover os NaNs.

**Sugestão**: pesquise sobre a função DataFrame.join(), pode ser muito útil no futuro.

Para fazer essa junção, vamos utilizar o nome do país para `index` já que é a variável comum nos três arquivos.

Mudando o índice do *co2*:

In [ ]:
co2_idx = co2_2010.set_index('CO2 per capita')
co2_idx.head() 
# Note que os índices numéricos sumiram, e que agora o nome do país está em negrito. 
# Essa mudança indica que os nomes agora são os índices de referência para fazer a junção das bases de dados.

Mudando o índice do *income*: 



In [ ]:
inco_idx = income_2010.set_index('GDP per capita')
inco_idx.head()

Mudando o índice do *landlocked*:

In [ ]:
land_idx = landlocked.set_index('Country')
land_idx.head()

___
# A junção das bases de dados deve ser feita duas-a-duas

As duas bases de dados possuem mesmo nome da variável `2010`. Logo é necessário renomear esses nomes dos `dataset´s` da esquerda (left) e da direita (right).



In [ ]:
df = co2_idx.join(inco_idx, how='inner', lsuffix='_co2', rsuffix='_income')
df.head()

Junção da terceira base de dados com o *dataframe* acima:

In [ ]:
data = df.join(land_idx, how="inner")

Por que no exemplo acima não foi necessário usar prefixos ou sufixos?

In [ ]:
data.head()

Vamos agora **salvar** o *dataframe* para tornar mais fácil continuar a análise depois, se necessário. 

Note que esse será um requisito do Projeto 1 - salvar um *dataframe* e usar o arquivo salvo para fazer a análise.

In [ ]:
#Salvando o DataFrame em um arquivo CSV (comma separated values)
data.to_csv( 'co2_income.csv', index=True)

In [ ]:
data.to_excel('co2_income.xlsx')

___
# Explorando os índices

Indexação: vamos ver como descobrir se a Argentina é landlocked

In [ ]:
data.loc["Argentina", "landlocked"]

In [ ]:
data["landlocked"]["Argentina"]

In [ ]:
data.landlocked["Argentina"]

___
# Atividade 1 

Veja no tutorial como fazer uma seleção só com os países da América do Norte.

* Atenção: Considere como América do Norte apenas Estados Unidos, Canadá e México.



In [ ]:
data.loc[["United States", "Canada", "Mexico"], :]

Podemos também selecionar pela variável `region` considerando a categoria `america_north`, mas essa <font color=red> contém países que na verdade são da América Central</font>.


In [ ]:
data[data.region=="america_north"]

___
#  Atividade 2

Qual o valor de *2010_income* que separa os $75\%$ mais ricos?  Monte um *dataframe* separado só com os $80\%$ mais ricos segundo a coluna *2010_income*?  

Dica: Veja as funções [quantile](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.quantile.html) e [percentile](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.percentile.html)

O `describe()` já traz esse valor que separa os 75% mais pobres. Esse valor é chamando de 3o. quartil, ou seja, Q3.

In [ ]:
data["2010_income"].describe()

Note que os 80% mais ricos ficam <font color=red>**a partir dos 20% mais pobres**</font>

In [ ]:
p20=data["2010_income"].quantile(0.2)
p20

Podemos também fazer usando a função `np.percentile`. Note que esta função espera um **percentil** de 0 a 100.

In [ ]:
p20=np.percentile(data["2010_income"], q = 20)
p20

In [ ]:
#Monte um *dataframe* separado só com os $80\%$ mais ricos segundo a coluna *2010_income*?  
data[data['2010_income']>p20]